<a href="https://colab.research.google.com/github/jford59/ML1/blob/main/HW6/ML6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import torch.nn as nn
from collections import OrderedDict

drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))

torch.set_printoptions(edgeitems=2, linewidth=75)

varlist =  ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price']
charlist = ['mainroad','guestroom','basement','hotwaterheating','airconditioning','prefarea']

def binary_map(x):
  return x.map({'yes' : 1, 'no' : 0})
housing[charlist] = housing[charlist].apply(binary_map)

scaler = preprocessing.MinMaxScaler()
housing[varlist] = scaler.fit_transform(housing[varlist])

housing.pop('furnishingstatus')
x = housing.values
y = housing.pop('price')

X = torch.tensor(x)
y = torch.tensor(y)

n_samples = x.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]


X_train = X[train_indices]
y_train = y[train_indices]

X_test = X[val_indices]
y_test = y[val_indices]

def training_loop(n_epochs, optimizer, model, loss_fn, X_train, X_test, y_train, y_test):
  for epoch in range(1, n_epochs + 1):
    t_p_train = model(X_train) # <1>
    loss_train = loss_fn(t_p_train, y_train)

    t_p_test = model(X_test) # <1>
    loss_val = loss_fn(t_p_test, y_test)

    optimizer.zero_grad()
    loss_train.backward() # <2>
    optimizer.step()

    if epoch % 500 == 0 :
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}," f" Validation loss {loss_val.item():.4f}")

def loss_fn(t_p_train, y):
    squared_diffs = (t_p_train - y)**2
    return squared_diffs.mean()

neuron_count = 32

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(12, neuron_count)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(neuron_count, 1))
]))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

#1 layer
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    X_train = X_train.to(torch.float32),
    X_test = X_test.to(torch.float32),
    y_train = y_train.to(torch.float32),
    y_test = y_test.to(torch.float32)
    )

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(12, 32)),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear1', nn.Linear(32, 64)),
    ('hidden_activation1', nn.Tanh()),
    ('hidden_linear2', nn.Linear(64, 16)),
    ('hidden_activation2', nn.Tanh()),
    ('output_linear', nn.Linear(16, 1))
]))

#3 layers
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    X_train = X_train.to(torch.float32),
    X_test = X_test.to(torch.float32),
    y_train = y_train.to(torch.float32),
    y_test = y_test.to(torch.float32))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training loss 0.0309, Validation loss 0.0193
Epoch 1000, Training loss 0.0304, Validation loss 0.0192
Epoch 1500, Training loss 0.0301, Validation loss 0.0189
Epoch 2000, Training loss 0.0298, Validation loss 0.0186
Epoch 2500, Training loss 0.0297, Validation loss 0.0184
Epoch 3000, Training loss 0.0295, Validation loss 0.0183
Epoch 3500, Training loss 0.0294, Validation loss 0.0182
Epoch 4000, Training loss 0.0293, Validation loss 0.0181
Epoch 4500, Training loss 0.0293, Validation loss 0.0180
Epoch 5000, Training loss 0.0292, Validation loss 0.0180
Epoch 500, Training loss 0.1013, Validation loss 0.0645
Epoch 1000, Training loss 0.1013, Validation loss 0.0645
Epoch 1500, Training loss 0.1013, Validation loss 0.0645
Epoch 2000, Training loss 0.1013, Validation loss 0.0645
Epoch 2500, Training loss 0.1013, Validation loss 0.0645
Epoch 3000, Training loss 0.1013, Validation loss 0.0645
Epoch 3500, Training loss 0.1013, Validation loss 0.0645
Epoch 4000, Training loss 0.1013,